# Proyecto ATP-Analisis-Ranking
## Descripción
Este proyecto tiene como objetivo analizar diversas estadísticas de la base de datos de la ATP (Asociación de Tenistas Profesionales) y construir un modelo personalizado de
ranking basado en el rendimiento de los jugadores. Posteriormente, compararemos nuestro ranking con el ranking actual de la ATP para determinar la efectividad de nuestro modelo.

In [1174]:
# IMPORT LIBRARIES

import pandas as pd
import numpy as np
import plotly.express as px

## Read Data

In [1175]:
df_serve = pd.read_csv('Serve _ATP_Tennis.csv')
df_return = pd.read_csv('return_ ATP_Tennis.csv')
df_underpressure = pd.read_csv('Underpressure_ATP.csv')
df_win_loss = pd.read_csv('Win_Loss_ ATP_Tennis.csv')
df_ranking = pd.read_csv('ATP_Ranking.csv')

## Data Preprocessing

In [1176]:
## DATA PREPROCESSING - df_serve

### Nos quedamos solo con las columnas que nos interesan.

df_serve = df_serve[["ServeRating©2", "_1stServe", "_1stServePointsWon", "_2ndServePointsWon", "_ServiceGamesWon", "Avg\.Aces_Match", "Avg\.DoubleFaults_Match", "Field"]]
df_serve.head()

,ServeRating©2,_1stServe,_1stServePointsWon,_2ndServePointsWon,_ServiceGamesWon,Avg\.Aces_Match,Avg\.DoubleFaults_Match,Field
0,Nick Kyrgios,308.6,67.4%,79.4%,57.6%,92.9%,14.8,3.5
1,John Isner,308.5,67.7%,79.8%,52.7%,90.2%,21.1,3.0
2,Hubert Hurkacz,299.7,63.4%,79.2%,54.9%,89.9%,14.0,1.7
3,Matteo Berrettini,292.2,62.5%,79.6%,52.0%,88.5%,11.5,1.9
4,Novak Djokovic,292.1,65.3%,77.2%,57.0%,88.7%,6.0,2.1


In [1177]:
### Hacemos un rename en el nombre de las columnas para que sean más intuitivas.

df_serve.rename(columns={"ServeRating©2": "Player", "_1stServe": "Serve Rating", "_1stServePointsWon": "%1stServe", "_2ndServePointsWon": "%1stServePointsWon", "_ServiceGamesWon": "%2ndServePointsWon", "Avg\.Aces_Match": "%ServiceGamesWon", "Avg\.DoubleFaults_Match": "AvgACESMatch", "Field": "AvgDoubleFaultsMatch"}, inplace=True)
df_serve.head()

,Player,Serve Rating,%1stServe,%1stServePointsWon,%2ndServePointsWon,%ServiceGamesWon,AvgACESMatch,AvgDoubleFaultsMatch
0,Nick Kyrgios,308.6,67.4%,79.4%,57.6%,92.9%,14.8,3.5
1,John Isner,308.5,67.7%,79.8%,52.7%,90.2%,21.1,3.0
2,Hubert Hurkacz,299.7,63.4%,79.2%,54.9%,89.9%,14.0,1.7
3,Matteo Berrettini,292.2,62.5%,79.6%,52.0%,88.5%,11.5,1.9
4,Novak Djokovic,292.1,65.3%,77.2%,57.0%,88.7%,6.0,2.1


In [1178]:
### Vamos a ver que tipo de datos tenemos en cada columna y si hay valores nulos.

df_serve.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Player                77 non-null     object 
 1   Serve Rating          77 non-null     float64
 2   %1stServe             77 non-null     object 
 3   %1stServePointsWon    77 non-null     object 
 4   %2ndServePointsWon    77 non-null     object 
 5   %ServiceGamesWon      77 non-null     object 
 6   AvgACESMatch          77 non-null     float64
 7   AvgDoubleFaultsMatch  77 non-null     float64
dtypes: float64(3), object(5)
memory usage: 4.9+ KB


Como podemos ver, tenemos 5 columnas con datos object y 3 columnas con datos Float64. Además, no tenemos valores nulos en ninguna columna.
Lo que vamos a hacer es cambiar todas las columnas con datos object a tipo float, excepto la primera columna que contiene el nombre del jugador.

In [1179]:
# Creamos una funcion que nos sirva para todos los dataframes.

def convert_to_float(df):
    """
    Esta función elimina el símbolo de porcentaje y convierte todos los valores en un DataFrame a tipo 'float',
    dividiendo por 100, excepto la primera columna.

    :param df: DataFrame de entrada
    :return: DataFrame con los valores modificados
    """
    for column in df.columns[1:]:
        df[column] = df[column].apply(lambda x: str(x).replace("%", "") if isinstance(x, str) else x)
        df[column] = pd.to_numeric(df[column], errors='coerce', downcast='float')

    return df



In [1180]:
# convertimos todas las columnas a float

df_serve = convert_to_float(df_serve)
df_serve.head()

,Player,Serve Rating,%1stServe,%1stServePointsWon,%2ndServePointsWon,%ServiceGamesWon,AvgACESMatch,AvgDoubleFaultsMatch
0,Nick Kyrgios,308.600006,67.400002,79.400002,57.599998,92.900002,14.8,3.5
1,John Isner,308.500000,67.699997,79.800003,52.700001,90.199997,21.1,3.0
2,Hubert Hurkacz,299.700012,63.400002,79.199997,54.900002,89.900002,14.0,1.7
3,Matteo Berrettini,292.200012,62.500000,79.599998,52.000000,88.500000,11.5,1.9
4,Novak Djokovic,292.100006,65.300003,77.199997,57.000000,88.699997,6.0,2.1


In [1181]:
# Comprobamos que los datos se han convertido correctamente.

df_serve.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Player                77 non-null     object 
 1   Serve Rating          77 non-null     float32
 2   %1stServe             77 non-null     float32
 3   %1stServePointsWon    77 non-null     float32
 4   %2ndServePointsWon    77 non-null     float32
 5   %ServiceGamesWon      77 non-null     float32
 6   AvgACESMatch          77 non-null     float32
 7   AvgDoubleFaultsMatch  77 non-null     float32
dtypes: float32(7), object(1)
memory usage: 2.8+ KB


In [1182]:
# DATA PREPROCESSING - df_return

### Nos quedamos solo con las columnas que nos interesan.

df_return = df_return[["ReturnRating©2", "_1stServeReturnPointsWon", "_2ndServeReturnPointsWon",  "_ReturnGamesWon", "_BreakPointsConverted", "Field"]]
df_return.head()

,ReturnRating©2,_1stServeReturnPointsWon,_2ndServeReturnPointsWon,_ReturnGamesWon,_BreakPointsConverted,Field
0,Bernabe Zapata Miralles,165.3,34.2%,52.3%,31.0%,47.8%
1,Daniil Medvedev,164.6,32.8%,55.2%,30.6%,46.0%
2,Rafael Nadal,160.8,34.4%,51.7%,30.0%,44.7%
3,Carlos Alcaraz,160.6,34.4%,53.5%,31.4%,41.3%
4,Novak Djokovic,160.5,33.5%,55.9%,29.3%,41.8%


In [1183]:
### Hacemos un rename en el nombre de las columnas para que sean más intuitivas.

df_return.rename(columns={"ReturnRating©2": "Player", "_1stServeReturnPointsWon": "Return Rating", "_2ndServeReturnPointsWon": "%1st Serve Return Points Won", "_ReturnGamesWon": "%2nd Serve Return Points Won", "_BreakPointsConverted": "%Return Games Won", "Field": "%Breaks Points Converted"}, inplace=True)
df_return.head()

,Player,Return Rating,%1st Serve Return Points Won,%2nd Serve Return Points Won,%Return Games Won,%Breaks Points Converted
0,Bernabe Zapata Miralles,165.3,34.2%,52.3%,31.0%,47.8%
1,Daniil Medvedev,164.6,32.8%,55.2%,30.6%,46.0%
2,Rafael Nadal,160.8,34.4%,51.7%,30.0%,44.7%
3,Carlos Alcaraz,160.6,34.4%,53.5%,31.4%,41.3%
4,Novak Djokovic,160.5,33.5%,55.9%,29.3%,41.8%


In [1184]:
### Vamos a ver que tipo de datos tenemos en cada columna y si hay valores nulos.

df_return.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Player                        77 non-null     object 
 1   Return Rating                 77 non-null     float64
 2   %1st Serve Return Points Won  77 non-null     object 
 3   %2nd Serve Return Points Won  77 non-null     object 
 4   %Return Games Won             77 non-null     object 
 5   %Breaks Points Converted      77 non-null     object 
dtypes: float64(1), object(5)
memory usage: 3.7+ KB


In [1185]:
# convertimos todas las columnas a float

df_return = convert_to_float(df_return)
df_return.head()

,Player,Return Rating,%1st Serve Return Points Won,%2nd Serve Return Points Won,%Return Games Won,%Breaks Points Converted
0,Bernabe Zapata Miralles,165.300003,34.200001,52.299999,31.000000,47.799999
1,Daniil Medvedev,164.600006,32.799999,55.200001,30.600000,46.000000
2,Rafael Nadal,160.800003,34.400002,51.700001,30.000000,44.700001
3,Carlos Alcaraz,160.600006,34.400002,53.500000,31.400000,41.299999
4,Novak Djokovic,160.500000,33.500000,55.900002,29.299999,41.799999


In [1186]:
# Comprobamos que los datos se han convertido correctamente.

df_return.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Player                        77 non-null     object 
 1   Return Rating                 77 non-null     float32
 2   %1st Serve Return Points Won  77 non-null     float32
 3   %2nd Serve Return Points Won  77 non-null     float32
 4   %Return Games Won             77 non-null     float32
 5   %Breaks Points Converted      77 non-null     float32
dtypes: float32(5), object(1)
memory usage: 2.2+ KB


In [1187]:
# DATA PREPROCESSING - df_underpressure

### Nos quedamos solo con las columnas que nos interesan.

df_underpressure = df_underpressure[["UnderPressureRating©2", "_BreakPointsConverted", "_BreakPointsSaved", "_TieBreaksWon", "_DecidingSetsWon", "Field"]]
df_underpressure.head()

,UnderPressureRating©2,_BreakPointsConverted,_BreakPointsSaved,_TieBreaksWon,_DecidingSetsWon,Field
0,Novak Djokovic,249.7,41.8%,64.3%,80.0%,63.6%
1,Nick Kyrgios,249.7,43.4%,74.4%,61.9%,70.0%
2,Jenson Brooksby,246.9,46.3%,58.9%,75.0%,66.7%
3,Jannik Sinner,238.6,43.3%,63.5%,68.2%,63.6%
4,Stefanos Tsitsipas,237.7,41.8%,66.0%,56.8%,73.1%


In [1188]:
### Hacemos un rename en el nombre de las columnas para que sean más intuitivas.

df_underpressure.rename(columns={"UnderPressureRating©2": "Player", "_BreakPointsConverted": "Underpressure Rating", "_BreakPointsSaved": "%BreakPointsConverted", "_TieBreaksWon": "%BreaksPointsSaved", "_DecidingSetsWon": "%TieBreaks Won", "Field": "%Deciding Sets Won"}, inplace=True)
df_underpressure.head()

,Player,Underpressure Rating,%BreakPointsConverted,%BreaksPointsSaved,%TieBreaks Won,%Deciding Sets Won
0,Novak Djokovic,249.7,41.8%,64.3%,80.0%,63.6%
1,Nick Kyrgios,249.7,43.4%,74.4%,61.9%,70.0%
2,Jenson Brooksby,246.9,46.3%,58.9%,75.0%,66.7%
3,Jannik Sinner,238.6,43.3%,63.5%,68.2%,63.6%
4,Stefanos Tsitsipas,237.7,41.8%,66.0%,56.8%,73.1%


In [1189]:
# Vamos a ver que tipo de datos tenemos en cada columna y si hay valores nulos.

df_underpressure.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Player                 77 non-null     object 
 1   Underpressure Rating   77 non-null     float64
 2   %BreakPointsConverted  77 non-null     object 
 3   %BreaksPointsSaved     77 non-null     object 
 4   %TieBreaks Won         77 non-null     object 
 5   %Deciding Sets Won     77 non-null     object 
dtypes: float64(1), object(5)
memory usage: 3.7+ KB


In [1190]:
# convertimos todas las columnas a float

df_underpressure = convert_to_float(df_underpressure)
df_underpressure.head()

,Player,Underpressure Rating,%BreakPointsConverted,%BreaksPointsSaved,%TieBreaks Won,%Deciding Sets Won
0,Novak Djokovic,249.699997,41.799999,64.300003,80.000000,63.599998
1,Nick Kyrgios,249.699997,43.400002,74.400002,61.900002,70.000000
2,Jenson Brooksby,246.899994,46.299999,58.900002,75.000000,66.699997
3,Jannik Sinner,238.600006,43.299999,63.500000,68.199997,63.599998
4,Stefanos Tsitsipas,237.699997,41.799999,66.000000,56.799999,73.099998


In [1191]:
# Comprobamos que los datos se han convertido correctamente.

df_underpressure.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Player                 77 non-null     object 
 1   Underpressure Rating   77 non-null     float32
 2   %BreakPointsConverted  77 non-null     float32
 3   %BreaksPointsSaved     77 non-null     float32
 4   %TieBreaks Won         77 non-null     float32
 5   %Deciding Sets Won     77 non-null     float32
dtypes: float32(5), object(1)
memory usage: 2.2+ KB


In [1192]:
# DATA PREPROCESSING - df_win_loss

### Nos quedamos solo con las columnas que nos interesan.

df_win_loss = df_win_loss[["YTDWin_Loss", "Field"]]
df_win_loss.head()

,YTDWin_Loss,Field
0,Novak Djokovic,0.938
1,Carlos Alcaraz,0.933
2,Daniil Medvedev,0.889
3,Cameron Norrie,0.840
4,Nicolas Jarry,0.833


In [1193]:
### Hacemos un rename en el nombre de las columnas para que sean más intuitivas.

df_win_loss.rename(columns={"YTDWin_Loss": "Player", "Field": "%AvgWinLoss"}, inplace=True)
df_win_loss.head()

,Player,%AvgWinLoss
0,Novak Djokovic,0.938
1,Carlos Alcaraz,0.933
2,Daniil Medvedev,0.889
3,Cameron Norrie,0.840
4,Nicolas Jarry,0.833


In [1194]:
# Vamos a ver que tipo de datos tenemos en cada columna y si hay valores nulos.

df_win_loss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Player       90 non-null     object 
 1   %AvgWinLoss  90 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.5+ KB


In [1195]:
# convertimos todas las columnas a float

df_win_loss = convert_to_float(df_win_loss)
df_win_loss.head()

,Player,%AvgWinLoss
0,Novak Djokovic,0.938
1,Carlos Alcaraz,0.933
2,Daniil Medvedev,0.889
3,Cameron Norrie,0.840
4,Nicolas Jarry,0.833


In [1196]:
# Comprobamos que los datos se han convertido correctamente.

df_win_loss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Player       90 non-null     object 
 1   %AvgWinLoss  90 non-null     float32
dtypes: float32(1), object(1)
memory usage: 1.2+ KB


In [1197]:
# DATA PREPROCESSING - df_ranking

### Nos quedamos solo con las columnas que nos interesan.

df_ranking = df_ranking[["Rank", "Player", "Age"]]

df_ranking.head()

,Rank,Player,Age
0,\n 1\n,\n Carlos Alcaraz\n...,\n19
1,\n 2\n,\n Novak Djokovic\n...,\n35
2,\n 3\n,\n Stefanos Tsitsip...,\n24
3,\n 4\n,\n Casper Ruud\n ...,\n24
4,\n 5\n,\n Daniil Medvedev\...,\n27


In [1198]:
# Vamos a ver que tipo de datos tenemos en cada columna y si hay valores nulos.

df_ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1998 entries, 0 to 1997
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rank    1998 non-null   object
 1   Player  1992 non-null   object
 2   Age     1992 non-null   object
dtypes: object(3)
memory usage: 47.0+ KB


In [1199]:
# Vamos a cambiar el orden de las columnas para que sea más intuitivo.

df_ranking = df_ranking[["Player", "Age", "Rank"]]
df_ranking.head()


,Player,Age,Rank
0,\n Carlos Alcaraz\n...,\n19,\n 1\n
1,\n Novak Djokovic\n...,\n35,\n 2\n
2,\n Stefanos Tsitsip...,\n24,\n 3\n
3,\n Casper Ruud\n ...,\n24,\n 4\n
4,\n Daniil Medvedev\...,\n27,\n 5\n


In [1200]:
# convertimos todas las columnas a float

df_ranking = convert_to_float(df_ranking)
df_ranking.head()

,Player,Age,Rank
0,\n Carlos Alcaraz\n...,19.0,1.0
1,\n Novak Djokovic\n...,35.0,2.0
2,\n Stefanos Tsitsip...,24.0,3.0
3,\n Casper Ruud\n ...,24.0,4.0
4,\n Daniil Medvedev\...,27.0,5.0


In [1201]:
# Por ultimo vamos a quitar todas las \n que hay en la columna Player.

df_ranking['Player'] = df_ranking['Player'].apply(lambda x: x.replace('\n', '') if isinstance(x, str) else x)
df_ranking.head()


,Player,Age,Rank
0,Carlos Alcaraz ...,19.0,1.0
1,Novak Djokovic ...,35.0,2.0
2,Stefanos Tsitsipas...,24.0,3.0
3,Casper Ruud ...,24.0,4.0
4,Daniil Medvedev ...,27.0,5.0


In [1202]:
# Comprobamos que los datos se han convertido correctamente.

df_ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1998 entries, 0 to 1997
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  1992 non-null   object 
 1   Age     1980 non-null   float32
 2   Rank    1188 non-null   float32
dtypes: float32(2), object(1)
memory usage: 31.3+ KB


# MERGE DATAFRAMES

## Comenzamos uniendo los 4 dataframes que contienen las estadísticas de los jugadores para crear un único dataframe.

In [1203]:
# Vamos a hacer un merge de los 4 dataframes que contienen las estadísticas de los jugadores.
# Usamos la columna Player como clave para hacer el merge.

# Primero la columna Player la pasamos a formato Title para que coincida con el formato de los otros dataframes.
# Eliminamos si tiene espacios al principio o al final.

df_ranking['Player'] = df_ranking['Player'].apply(lambda x: x.title().strip() if isinstance(x, str) else x)
df_ranking.head()

,Player,Age,Rank
0,Carlos Alcaraz,19.0,1.0
1,Novak Djokovic,35.0,2.0
2,Stefanos Tsitsipas,24.0,3.0
3,Casper Ruud,24.0,4.0
4,Daniil Medvedev,27.0,5.0


In [1204]:
# Hacemos el merge de los 4 dataframes.

df_stats = pd.merge(df_serve, df_return, on='Player')
df_stats = pd.merge(df_stats, df_underpressure, on='Player')
df_stats = pd.merge(df_stats, df_win_loss, on='Player')
df_stats.head()

,Player,Serve Rating,%1stServe,%1stServePointsWon,%2ndServePointsWon,%ServiceGamesWon,AvgACESMatch,AvgDoubleFaultsMatch,Return Rating,%1st Serve Return Points Won,%2nd Serve Return Points Won,%Return Games Won,%Breaks Points Converted,Underpressure Rating,%BreakPointsConverted,%BreaksPointsSaved,%TieBreaks Won,%Deciding Sets Won,%AvgWinLoss
0,John Isner,308.500000,67.699997,79.800003,52.700001,90.199997,21.1,3.0,93.800003,20.900000,40.000000,7.100000,25.799999,219.100006,25.799999,69.300003,71.099998,52.900002,0.444
1,Hubert Hurkacz,299.700012,63.400002,79.199997,54.900002,89.900002,14.0,1.7,125.300003,27.200001,46.599998,16.500000,35.000000,226.399994,35.000000,67.699997,54.700001,69.000000,0.684
2,Matteo Berrettini,292.200012,62.500000,79.599998,52.000000,88.500000,11.5,1.9,129.800003,29.500000,45.700001,18.799999,35.799999,210.000000,35.799999,67.099998,50.000000,57.099998,0.500
3,Novak Djokovic,292.100006,65.300003,77.199997,57.000000,88.699997,6.0,2.1,160.500000,33.500000,55.900002,29.299999,41.799999,249.699997,41.799999,64.300003,80.000000,63.599998,0.938
4,Ben Shelton,292.000000,60.700001,79.300003,55.900002,89.400002,11.3,4.6,118.500000,23.799999,42.099998,13.300000,39.299999,209.000000,39.299999,70.500000,56.299999,42.900002,0.545


In [1205]:
# hacemos lo mismo con el dataframe de ranking.

df_stats['Player'] = df_stats['Player'].apply(lambda x: x.title().strip() if isinstance(x, str) else x)
df_stats.head()

,Player,Serve Rating,%1stServe,%1stServePointsWon,%2ndServePointsWon,%ServiceGamesWon,AvgACESMatch,AvgDoubleFaultsMatch,Return Rating,%1st Serve Return Points Won,%2nd Serve Return Points Won,%Return Games Won,%Breaks Points Converted,Underpressure Rating,%BreakPointsConverted,%BreaksPointsSaved,%TieBreaks Won,%Deciding Sets Won,%AvgWinLoss
0,John Isner,308.500000,67.699997,79.800003,52.700001,90.199997,21.1,3.0,93.800003,20.900000,40.000000,7.100000,25.799999,219.100006,25.799999,69.300003,71.099998,52.900002,0.444
1,Hubert Hurkacz,299.700012,63.400002,79.199997,54.900002,89.900002,14.0,1.7,125.300003,27.200001,46.599998,16.500000,35.000000,226.399994,35.000000,67.699997,54.700001,69.000000,0.684
2,Matteo Berrettini,292.200012,62.500000,79.599998,52.000000,88.500000,11.5,1.9,129.800003,29.500000,45.700001,18.799999,35.799999,210.000000,35.799999,67.099998,50.000000,57.099998,0.500
3,Novak Djokovic,292.100006,65.300003,77.199997,57.000000,88.699997,6.0,2.1,160.500000,33.500000,55.900002,29.299999,41.799999,249.699997,41.799999,64.300003,80.000000,63.599998,0.938
4,Ben Shelton,292.000000,60.700001,79.300003,55.900002,89.400002,11.3,4.6,118.500000,23.799999,42.099998,13.300000,39.299999,209.000000,39.299999,70.500000,56.299999,42.900002,0.545


In [1206]:
# Hacemos el merge de los 2 dataframes.

df = pd.merge(df_stats, df_ranking, on='Player')
df.head()

,Player,Serve Rating,%1stServe,%1stServePointsWon,%2ndServePointsWon,%ServiceGamesWon,AvgACESMatch,AvgDoubleFaultsMatch,Return Rating,%1st Serve Return Points Won,...,%Return Games Won,%Breaks Points Converted,Underpressure Rating,%BreakPointsConverted,%BreaksPointsSaved,%TieBreaks Won,%Deciding Sets Won,%AvgWinLoss,Age,Rank
0,John Isner,308.500000,67.699997,79.800003,52.700001,90.199997,21.1,3.0,93.800003,20.900000,...,7.100000,25.799999,219.100006,25.799999,69.300003,71.099998,52.900002,0.444,37.0,46.0
1,Hubert Hurkacz,299.700012,63.400002,79.199997,54.900002,89.900002,14.0,1.7,125.300003,27.200001,...,16.500000,35.000000,226.399994,35.000000,67.699997,54.700001,69.000000,0.684,26.0,9.0
2,Matteo Berrettini,292.200012,62.500000,79.599998,52.000000,88.500000,11.5,1.9,129.800003,29.500000,...,18.799999,35.799999,210.000000,35.799999,67.099998,50.000000,57.099998,0.500,26.0,23.0
3,Novak Djokovic,292.100006,65.300003,77.199997,57.000000,88.699997,6.0,2.1,160.500000,33.500000,...,29.299999,41.799999,249.699997,41.799999,64.300003,80.000000,63.599998,0.938,35.0,2.0
4,Ben Shelton,292.000000,60.700001,79.300003,55.900002,89.400002,11.3,4.6,118.500000,23.799999,...,13.300000,39.299999,209.000000,39.299999,70.500000,56.299999,42.900002,0.545,20.0,39.0


# EDA - Exploratory Data Analysis

### Vamos a averiguar que jugadores tienen un %1stServePointsWon superior a la media.

In [1207]:
### Vamos a comenzar analizando la columna %1stServePointsWon por cada Player.

Best1stServePointsWon = df[['Player', '%1stServePointsWon']]
Best1stServePointsWon.head()

,Player,%1stServePointsWon
0,John Isner,79.800003
1,Hubert Hurkacz,79.199997
2,Matteo Berrettini,79.599998
3,Novak Djokovic,77.199997
4,Ben Shelton,79.300003


In [1208]:
# Vamos a ver la distribución de los datos.

Best1stServePointsWon.describe()

,%1stServePointsWon
count,68.000000
mean,71.688232
std,4.551350
min,61.799999
25%,68.500000
50%,71.850002
75%,75.249998
max,79.800003


In [1209]:
# Vamos a ver la distribución de los datos con plotly con forma de linea.

fig = px.line(Best1stServePointsWon, x='Player', y='%1stServePointsWon', title='1stServePointsWon')
fig.show()

In [1210]:
# vamos a establecer los cuartiles para ver que jugadores están por encima de la media.

q1 = Best1stServePointsWon['%1stServePointsWon'].quantile(0.25)
q3 = Best1stServePointsWon['%1stServePointsWon'].quantile(0.75)
iqr = q3 - q1 #Interquartile range
fence_low  = q1-1.5*iqr #lower fence. Esto es para ver los valores que están por debajo del primer cuartil.
fence_high = q3+1.5*iqr #upper fence. Esto es para ver los valores que están por encima del tercer cuartil.

print('q1: ', q1)
print('q3: ', q3)
print('iqr: ', iqr)
print('fence_low: ', fence_low)
print('fence_high: ', fence_high)

q1:  68.5
q3:  75.24999809265137
iqr:  6.749998092651367
fence_low:  58.37500286102295
fence_high:  85.37499523162842


In [1211]:
### Vamos a ver los jugadores que están por encima de la media.

Best1stServePointsWon = Best1stServePointsWon[Best1stServePointsWon['%1stServePointsWon'] > q3]
Best1stServePointsWon

,Player,%1stServePointsWon
0,John Isner,79.800003
1,Hubert Hurkacz,79.199997
2,Matteo Berrettini,79.599998
3,Novak Djokovic,77.199997
4,Ben Shelton,79.300003
5,Stefanos Tsitsipas,76.500000
6,Maxime Cressy,79.800003
7,Felix Auger-Aliassime,77.500000
8,Taylor Fritz,77.099998
10,Marc-Andrea Huesler,76.000000


### Vamos a averiguar que jugadores tienen un %2stServePointsWon superior a la media.

In [1212]:
### Hagamos lo mismo %2ndServePointsWon

Best2stServePointsWon = df[['Player', '%2ndServePointsWon']]
Best2stServePointsWon.head()

,Player,%2ndServePointsWon
0,John Isner,52.700001
1,Hubert Hurkacz,54.900002
2,Matteo Berrettini,52.000000
3,Novak Djokovic,57.000000
4,Ben Shelton,55.900002


In [1213]:
### Vamos a ver la distribución de los datos.

Best2stServePointsWon.describe()

,%2ndServePointsWon
count,68.000000
mean,51.483822
std,2.677246
min,44.099998
25%,50.175000
50%,51.400002
75%,53.025000
max,57.000000


In [1214]:
### Vamos a ver la distribución de los datos con plotly con forma de linea.

fig = px.line(Best2stServePointsWon, x='Player', y='%2ndServePointsWon', title='2ndServePointsWon')
fig.show()

In [1215]:
### vamos a establecer los cuartiles para ver que jugadores están por encima de la media.

q1 = Best2stServePointsWon['%2ndServePointsWon'].quantile(0.25)
q3 = Best2stServePointsWon['%2ndServePointsWon'].quantile(0.75)
iqr = q3 - q1 #Interquartile range
fence_low  = q1-1.5*iqr #lower fence. Esto es para ver los valores que están por debajo del primer cuartil.
fence_high = q3+1.5*iqr #upper fence. Esto es para ver los valores que están por encima del tercer cuartil.

print('q1: ', q1)
print('q3: ', q3)
print('iqr: ', iqr)
print('fence_low: ', fence_low)
print('fence_high: ', fence_high)

q1:  50.17500019073486
q3:  53.02499961853027
iqr:  2.84999942779541
fence_low:  45.90000104904175
fence_high:  57.29999876022339


In [1216]:
### Vamos a ver los jugadores que están por encima de la media.

Best2stServePointsWon = Best2stServePointsWon[Best2stServePointsWon['%2ndServePointsWon'] > q3]
Best2stServePointsWon

,Player,%2ndServePointsWon
1,Hubert Hurkacz,54.900002
3,Novak Djokovic,57.000000
4,Ben Shelton,55.900002
5,Stefanos Tsitsipas,55.099998
6,Maxime Cressy,53.099998
9,Casper Ruud,55.200001
12,Borna Coric,54.500000
14,Holger Rune,55.099998
15,Carlos Alcaraz,55.299999
20,Roberto Bautista Agut,55.099998


### Vamos a averiguar que jugadores tienen un AvgACESMatch superior a la media.

In [1217]:
### Hagamos lo mismo AvgACESMatch

BestAvgACESMatch = df[['Player', 'AvgACESMatch']]
BestAvgACESMatch.head()

,Player,AvgACESMatch
0,John Isner,21.1
1,Hubert Hurkacz,14.0
2,Matteo Berrettini,11.5
3,Novak Djokovic,6.0
4,Ben Shelton,11.3


In [1218]:
### Vamos a ver la distribución de los datos.

BestAvgACESMatch.describe()

,AvgACESMatch
count,68.000000
mean,6.022059
std,3.513082
min,1.000000
25%,3.325000
50%,5.250000
75%,7.800000
max,21.100000


In [1219]:
### Vamos a ver la distribución de los datos con plotly con forma de linea.

fig = px.line(BestAvgACESMatch, x='Player', y='AvgACESMatch', title='AvgACESMatch')
fig.show()

In [1220]:
### vamos a establecer los cuartiles para ver que jugadores están por encima de la media.

q1 = BestAvgACESMatch['AvgACESMatch'].quantile(0.25)
q3 = BestAvgACESMatch['AvgACESMatch'].quantile(0.75)
iqr = q3 - q1 #Interquartile range
fence_low  = q1-1.5*iqr #lower fence. Esto es para ver los valores que están por debajo del primer cuartil.
fence_high = q3+1.5*iqr #upper fence. Esto es para ver los valores que están por encima del tercer cuartil.

print('q1: ', q1)
print('q3: ', q3)
print('iqr: ', iqr)
print('fence_low: ', fence_low)
print('fence_high: ', fence_high)

q1:  3.325000047683716
q3:  7.800000190734863
iqr:  4.4750001430511475
fence_low:  -3.3875001668930054
fence_high:  14.512500405311584


In [1221]:
### Vamos a ver los jugadores que están por encima de la media.

BestAvgACESMatch = BestAvgACESMatch[BestAvgACESMatch['AvgACESMatch'] > q3]
BestAvgACESMatch

,Player,AvgACESMatch
0,John Isner,21.1
1,Hubert Hurkacz,14.0
2,Matteo Berrettini,11.5
4,Ben Shelton,11.3
6,Maxime Cressy,15.2
7,Felix Auger-Aliassime,10.5
8,Taylor Fritz,9.2
10,Marc-Andrea Huesler,8.8
21,Jack Draper,8.1
24,Frances Tiafoe,8.5


In [1222]:
### Vamos a comenzar analizando la columna %ServeGamesWon por cada Player.

BestServeGamesWon = df[['Player', '%ServiceGamesWon']]
BestServeGamesWon.head()

,Player,%ServiceGamesWon
0,John Isner,90.199997
1,Hubert Hurkacz,89.900002
2,Matteo Berrettini,88.500000
3,Novak Djokovic,88.699997
4,Ben Shelton,89.400002


In [1223]:
### Vamos a ver la distribución de los datos.

BestServeGamesWon.describe()

,%ServiceGamesWon
count,68.000000
mean,79.858826
std,5.575459
min,67.500000
25%,76.450001
50%,80.650002
75%,83.325003
max,90.199997


In [1224]:
### Vamos a ver la distribución de los datos con plotly con forma de linea.

fig = px.line(BestServeGamesWon, x='Player', y='%ServiceGamesWon', title='%ServiceGamesWon')
fig.show()

In [1225]:
### vamos a establecer los cuartiles para ver que jugadores están por encima de la media.

q1 = BestServeGamesWon['%ServiceGamesWon'].quantile(0.25)
q3 = BestServeGamesWon['%ServiceGamesWon'].quantile(0.75)
iqr = q3 - q1 #Interquartile range
fence_low  = q1-1.5*iqr #lower fence. Esto es para ver los valores que están por debajo del primer cuartil.
fence_high = q3+1.5*iqr #upper fence. Esto es para ver los valores que están por encima del tercer cuartil.

print('q1: ', q1)
print('q3: ', q3)
print('iqr: ', iqr)
print('fence_low: ', fence_low)
print('fence_high: ', fence_high)

q1:  76.45000076293945
q3:  83.32500267028809
iqr:  6.875001907348633
fence_low:  66.1374979019165
fence_high:  93.63750553131104


In [1226]:
### Vamos a ver los jugadores que están por encima de la media.

BestServeGamesWon = BestServeGamesWon[BestServeGamesWon['%ServiceGamesWon'] > q3]
BestServeGamesWon

,Player,%ServiceGamesWon
0,John Isner,90.199997
1,Hubert Hurkacz,89.900002
2,Matteo Berrettini,88.500000
3,Novak Djokovic,88.699997
4,Ben Shelton,89.400002
5,Stefanos Tsitsipas,86.800003
6,Maxime Cressy,88.599998
7,Felix Auger-Aliassime,86.500000
8,Taylor Fritz,85.500000
9,Casper Ruud,84.599998


In [1227]:
### Vamos a comenzar analizando la columna %AVGDoubleFaultsMatch por cada Player.

BestAVGDoubleFaultsMatch = df[['Player', 'AvgDoubleFaultsMatch']]
BestAVGDoubleFaultsMatch.head()

,Player,AvgDoubleFaultsMatch
0,John Isner,3.0
1,Hubert Hurkacz,1.7
2,Matteo Berrettini,1.9
3,Novak Djokovic,2.1
4,Ben Shelton,4.6


In [1228]:
### Vamos a ver la distribución de los datos.

BestAVGDoubleFaultsMatch.describe()

,AvgDoubleFaultsMatch
count,68.000000
mean,2.748530
std,1.238137
min,1.400000
25%,1.975000
50%,2.500000
75%,3.000000
max,9.000000


In [1229]:
### Vamos a ver la distribución de los datos con plotly con forma de linea.

fig = px.line(BestAVGDoubleFaultsMatch, x='Player', y='AvgDoubleFaultsMatch', title='AvgDoubleFaultsMatch')
fig.show()

In [1230]:
### vamos a establecer los cuartiles para ver que jugadores están por encima de la media.

q1 = BestAVGDoubleFaultsMatch['AvgDoubleFaultsMatch'].quantile(0.25)
q3 = BestAVGDoubleFaultsMatch['AvgDoubleFaultsMatch'].quantile(0.75)
iqr = q3 - q1 #Interquartile range
fence_low  = q1-1.5*iqr #lower fence. Esto es para ver los valores que están por debajo del primer cuartil.
fence_high = q3+1.5*iqr #upper fence. Esto es para ver los valores que están por encima del tercer cuartil.

print('q1: ', q1)
print('q3: ', q3)
print('iqr: ', iqr)
print('fence_low: ', fence_low)
print('fence_high: ', fence_high)

q1:  1.9749999940395355
q3:  3.0
iqr:  1.0250000059604645
fence_low:  0.4374999850988388
fence_high:  4.537500008940697


In [1231]:
### Vamos a ver los jugadores que están por encima de la media.

BestAVGDoubleFaultsMatch = BestAVGDoubleFaultsMatch[BestAVGDoubleFaultsMatch['AvgDoubleFaultsMatch'] < q1]
BestAVGDoubleFaultsMatch

,Player,AvgDoubleFaultsMatch
1,Hubert Hurkacz,1.7
2,Matteo Berrettini,1.9
8,Taylor Fritz,1.9
10,Marc-Andrea Huesler,1.9
12,Borna Coric,1.6
13,Lorenzo Sonego,1.7
20,Roberto Bautista Agut,1.8
26,Stan Wawrinka,1.7
28,Adrian Mannarino,1.8
34,Lorenzo Musetti,1.8


In [1232]:
### Vamos a comenzar analizando la columna %Return Games Won por cada Player.

BestReturnGamesWon = df[['Player', '%Return Games Won']]
BestReturnGamesWon.head()

,Player,%Return Games Won
0,John Isner,7.100000
1,Hubert Hurkacz,16.500000
2,Matteo Berrettini,18.799999
3,Novak Djokovic,29.299999
4,Ben Shelton,13.300000


In [1233]:
### Vamos a ver la distribución de los datos.

BestReturnGamesWon.describe()

,%Return Games Won
count,68.000000
mean,21.985294
std,4.839645
min,7.100000
25%,18.950000
50%,22.050000
75%,25.150000
max,31.400000


In [1234]:
### Vamos a ver la distribución de los datos con plotly con forma de linea.

fig = px.line(BestReturnGamesWon, x='Player', y='%Return Games Won', title='%Return Games Won')
fig.show()

In [1235]:
### vamos a establecer los cuartiles para ver que jugadores están por encima de la media.

q1 = BestReturnGamesWon['%Return Games Won'].quantile(0.25)
q3 = BestReturnGamesWon['%Return Games Won'].quantile(0.75)
iqr = q3 - q1 #Interquartile range
fence_low  = q1-1.5*iqr #lower fence. Esto es para ver los valores que están por debajo del primer cuartil.
fence_high = q3+1.5*iqr #upper fence. Esto es para ver los valores que están por encima del tercer cuartil.

print('q1: ', q1)
print('q3: ', q3)
print('iqr: ', iqr)
print('fence_low: ', fence_low)
print('fence_high: ', fence_high)

q1:  18.949999809265137
q3:  25.15000009536743
iqr:  6.200000286102295
fence_low:  9.649999380111694
fence_high:  34.450000524520874


In [1236]:
### Vamos a ver los jugadores que están por encima de la media.

BestReturnGamesWon = BestReturnGamesWon[BestReturnGamesWon['%Return Games Won'] > q3]
BestReturnGamesWon

,Player,%Return Games Won
3,Novak Djokovic,29.299999
11,Daniil Medvedev,30.600000
15,Carlos Alcaraz,31.400000
21,Jack Draper,25.400000
27,Jannik Sinner,27.700001
31,Cameron Norrie,27.000000
34,Lorenzo Musetti,25.400000
43,Mackenzie Mcdonald,25.299999
52,Alejandro Davidovich Fokina,27.700001
56,Alex De Minaur,29.400000


In [1237]:
### Vamos a comenzar analizando la columna %TieBreaks por cada Player.

BestTieBreaks = df[['Player', '%TieBreaks Won']]
BestTieBreaks.head()

,Player,%TieBreaks Won
0,John Isner,71.099998
1,Hubert Hurkacz,54.700001
2,Matteo Berrettini,50.000000
3,Novak Djokovic,80.000000
4,Ben Shelton,56.299999


In [1238]:
### Vamos a ver la distribución de los datos.

BestTieBreaks.describe()

,%TieBreaks Won
count,68.000000
mean,50.148529
std,13.331026
min,10.000000
25%,42.700001
50%,51.950001
75%,58.349999
max,80.000000


In [1239]:
### Vamos a ver la distribución de los datos con plotly con forma de linea.

fig = px.line(BestTieBreaks, x='Player', y='%TieBreaks Won', title='%TieBreaks Won')
fig.show()

In [1240]:
### vamos a establecer los cuartiles para ver que jugadores están por encima de la media.

q1 = BestTieBreaks['%TieBreaks Won'].quantile(0.25)
q3 = BestTieBreaks['%TieBreaks Won'].quantile(0.75)
iqr = q3 - q1 #Interquartile range
fence_low  = q1-1.5*iqr #lower fence. Esto es para ver los valores que están por debajo del primer cuartil.
fence_high = q3+1.5*iqr #upper fence. Esto es para ver los valores que están por encima del tercer cuartil.

print('q1: ', q1)
print('q3: ', q3)
print('iqr: ', iqr)
print('fence_low: ', fence_low)
print('fence_high: ', fence_high)

q1:  42.70000076293945
q3:  58.34999942779541
iqr:  15.649998664855957
fence_low:  19.225002765655518
fence_high:  81.82499742507935


In [1241]:
### Vamos a ver los jugadores que están por encima de la media.

BestTieBreaks = BestTieBreaks[BestTieBreaks['%TieBreaks Won'] > q3]
BestTieBreaks

,Player,%TieBreaks Won
0,John Isner,71.099998
3,Novak Djokovic,80.000000
8,Taylor Fritz,58.500000
9,Casper Ruud,61.000000
12,Borna Coric,66.699997
14,Holger Rune,63.599998
16,Alexander Zverev,78.599998
18,Andrey Rublev,61.500000
24,Frances Tiafoe,66.699997
27,Jannik Sinner,68.199997


In [1242]:
### Vamos a comenzar analizando la columna %Deciding Set Won por cada Player.

BestDecidingSetWon = df[['Player', '%Deciding Sets Won']]
BestDecidingSetWon.head()

,Player,%Deciding Sets Won
0,John Isner,52.900002
1,Hubert Hurkacz,69.000000
2,Matteo Berrettini,57.099998
3,Novak Djokovic,63.599998
4,Ben Shelton,42.900002


In [1243]:
### Vamos a ver la distribución de los datos.

BestDecidingSetWon.describe()

,%Deciding Sets Won
count,68.000000
mean,52.380886
std,14.477096
min,6.700000
25%,44.025002
50%,52.900002
75%,62.325001
max,83.300003


In [1244]:
### Vamos a ver la distribución de los datos con plotly con forma de linea.

fig = px.line(BestDecidingSetWon, x='Player', y='%Deciding Sets Won', title='%Deciding Sets Won')
fig.show()

In [1245]:
### vamos a establecer los cuartiles para ver que jugadores están por encima de la media.

q1 = BestDecidingSetWon['%Deciding Sets Won'].quantile(0.25)
q3 = BestDecidingSetWon['%Deciding Sets Won'].quantile(0.75)
iqr = q3 - q1 #Interquartile range
fence_low  = q1-1.5*iqr #lower fence. Esto es para ver los valores que están por debajo del primer cuartil.
fence_high = q3+1.5*iqr #upper fence. Esto es para ver los valores que están por encima del tercer cuartil.

print('q1: ', q1)
print('q3: ', q3)
print('iqr: ', iqr)
print('fence_low: ', fence_low)
print('fence_high: ', fence_high)

q1:  44.025001525878906
q3:  62.32500076293945
iqr:  18.299999237060547
fence_low:  16.575002670288086
fence_high:  89.77499961853027


In [1246]:
### Vamos a ver los jugadores que están por encima de la media.

BestDecidingSetWon = BestDecidingSetWon[BestDecidingSetWon['%Deciding Sets Won'] > q3]
BestDecidingSetWon

,Player,%Deciding Sets Won
1,Hubert Hurkacz,69.000000
3,Novak Djokovic,63.599998
5,Stefanos Tsitsipas,73.099998
7,Felix Auger-Aliassime,66.699997
15,Carlos Alcaraz,73.900002
18,Andrey Rublev,65.199997
22,Karen Khachanov,64.699997
23,Arthur Rinderknech,75.000000
27,Jannik Sinner,63.599998
31,Cameron Norrie,71.000000


## Ya tenemos los jugadores que tienen un buen rendimiento en cada una de las columnas.

# OBJETIVES:

## 1. Que grupos son las que más correlacion tienen con en el ranking de los jugadores.
## 2. Que jugadores estan en todas las columnas.
## 3. Cuanto influye la edad en el ranking de los jugadores.

In [1247]:
### 1.0 Vamos a ver de los grupos creados que grupos son las que más correlacion tienen con en el ranking de los jugadores.

#Best1stServePointsWon
#BestReturnGamesWon
#BestTieBreaks
#BestDecidingSetWon



In [1248]:
## 1.1 Vamos a ver la correlación de Best1stServePointsWon con el el ranking de los jugadores de df_ranking.

# Lo primero que hacemos es añadir la columna ranking a Best1stServePointsWon para poder compararla con el ranking de los jugadores.
# El primero tiene que ser el que tenga mayor valor en la columna %1st Serve Points Won.

Best1stServePointsWon = Best1stServePointsWon.reset_index(drop=True)
Best1stServePointsWon['Rank'] = Best1stServePointsWon.index + 1
Best1stServePointsWon

,Player,%1stServePointsWon,Rank
0,John Isner,79.800003,1
1,Hubert Hurkacz,79.199997,2
2,Matteo Berrettini,79.599998,3
3,Novak Djokovic,77.199997,4
4,Ben Shelton,79.300003,5
5,Stefanos Tsitsipas,76.500000,6
6,Maxime Cressy,79.800003,7
7,Felix Auger-Aliassime,77.500000,8
8,Taylor Fritz,77.099998,9
9,Marc-Andrea Huesler,76.000000,10


In [1249]:
# Vamos a ver la correlación de Best1stServePointsWon con el el ranking de los jugadores de df_ranking viendo si coincide en el ranking de cada jugador en ambos dataframes.

df_ranking = df_ranking.reset_index(drop=True)
df_ranking['Rank'] = df_ranking.index + 1
df_ranking


,Player,Age,Rank
0,Carlos Alcaraz,19.0,1
1,Novak Djokovic,35.0,2
2,Stefanos Tsitsipas,24.0,3
3,Casper Ruud,24.0,4
4,Daniil Medvedev,27.0,5
...,...,...,...
1993,Stefan Vujic,24.0,1994
1994,Raffael Schaer,20.0,1995
1995,Biagio Gramaticopolo,20.0,1996
1996,Akram El Sallaly,25.0,1997


In [1250]:
# Para ver cuanto % de correlación hay entre las dos columnas vamos a hacer un merge de los dos dataframes.

df_merge = pd.merge(Best1stServePointsWon, df_ranking, on='Player')
df_merge


,Player,%1stServePointsWon,Rank_x,Age,Rank_y
0,John Isner,79.800003,1,37.0,46
1,Hubert Hurkacz,79.199997,2,26.0,9
2,Matteo Berrettini,79.599998,3,26.0,23
3,Novak Djokovic,77.199997,4,35.0,2
4,Ben Shelton,79.300003,5,20.0,39
5,Stefanos Tsitsipas,76.500000,6,24.0,3
6,Maxime Cressy,79.800003,7,25.0,37
7,Felix Auger-Aliassime,77.500000,8,22.0,6
8,Taylor Fritz,77.099998,9,25.0,10
9,Marc-Andrea Huesler,76.000000,10,26.0,47


In [1251]:
# Nos quedamos solo con aquellos que estan entre los 10 primeros en el df_merge.

Best_players_1st_serve = df_merge[df_merge['Rank_y'] <= 10]
Best_players_1st_serve

,Player,%1stServePointsWon,Rank_x,Age,Rank_y
1,Hubert Hurkacz,79.199997,2,26.0,9
3,Novak Djokovic,77.199997,4,35.0,2
5,Stefanos Tsitsipas,76.500000,6,24.0,3
7,Felix Auger-Aliassime,77.500000,8,22.0,6
8,Taylor Fritz,77.099998,9,25.0,10
10,Daniil Medvedev,75.400002,11,27.0,5
11,Andrey Rublev,76.099998,12,25.0,7


In [1252]:
### Ya sabemos cuantos jugadores hay en el top 10 del dataframe Best1stServePointsWon que estan en el top 10 del ranking de los jugadores. Vamos a ver el % de coincidencia.

correlacion_1st_serve_won = Best_players_1st_serve['Rank_x'].corr(Best_players_1st_serve['Rank_y'])
print('La correlación entre el ranking de los jugadores y el %1st Serve Points Won es del: ', correlacion_1st_serve_won)

La correlación entre el ranking de los jugadores y el %1st Serve Points Won es del:  0.13978892276586047


In [1253]:
## 1.2 Vamos a ver la correlación de BestReturnGamesWon con el el ranking de los jugadores de df_ranking.

# Lo primero que hacemos es añadir la columna ranking a BestReturnGamesWon para poder compararla con el ranking de los jugadores.

BestReturnGamesWon = BestReturnGamesWon.reset_index(drop=True)
BestReturnGamesWon['Rank'] = BestReturnGamesWon.index + 1
BestReturnGamesWon

,Player,%Return Games Won,Rank
0,Novak Djokovic,29.299999,1
1,Daniil Medvedev,30.600000,2
2,Carlos Alcaraz,31.400000,3
3,Jack Draper,25.400000,4
4,Jannik Sinner,27.700001,5
5,Cameron Norrie,27.000000,6
6,Lorenzo Musetti,25.400000,7
7,Mackenzie Mcdonald,25.299999,8
8,Alejandro Davidovich Fokina,27.700001,9
9,Alex De Minaur,29.400000,10


In [1254]:
# Vamos a ver la correlación de BestReturnGamesWon con el el ranking de los jugadores de df_ranking viendo si coincide en el ranking de cada jugador en ambos dataframes.

df_ranking = df_ranking.reset_index(drop=True)
df_ranking['Rank'] = df_ranking.index + 1
df_ranking

,Player,Age,Rank
0,Carlos Alcaraz,19.0,1
1,Novak Djokovic,35.0,2
2,Stefanos Tsitsipas,24.0,3
3,Casper Ruud,24.0,4
4,Daniil Medvedev,27.0,5
...,...,...,...
1993,Stefan Vujic,24.0,1994
1994,Raffael Schaer,20.0,1995
1995,Biagio Gramaticopolo,20.0,1996
1996,Akram El Sallaly,25.0,1997


In [1255]:
# Para ver cuanto % de correlación hay entre las dos columnas vamos a hacer un merge de los dos dataframes.

df_merge = pd.merge(BestReturnGamesWon, df_ranking, on='Player')
df_merge

,Player,%Return Games Won,Rank_x,Age,Rank_y
0,Novak Djokovic,29.299999,1,35.0,2
1,Daniil Medvedev,30.600000,2,27.0,5
2,Carlos Alcaraz,31.400000,3,19.0,1
3,Jack Draper,25.400000,4,21.0,43
4,Jannik Sinner,27.700001,5,21.0,11
5,Cameron Norrie,27.000000,6,27.0,12
6,Lorenzo Musetti,25.400000,7,21.0,21
7,Mackenzie Mcdonald,25.299999,8,27.0,55
8,Alejandro Davidovich Fokina,27.700001,9,23.0,25
9,Alex De Minaur,29.400000,10,24.0,18


In [1256]:
# Nos quedamos solo con aquellos que estan entre los 10 primeros en el df_merge.

Best_players_return_games_won = df_merge[df_merge['Rank_y'] <= 10]
Best_players_return_games_won

,Player,%Return Games Won,Rank_x,Age,Rank_y
0,Novak Djokovic,29.299999,1,35.0,2
1,Daniil Medvedev,30.600000,2,27.0,5
2,Carlos Alcaraz,31.400000,3,19.0,1


In [1257]:
### Ya sabemos cuantos jugadores hay en el top 10 del dataframe BestReturnGamesWon que estan en el top 10 del ranking de los jugadores. Vamos a ver el % de coincidencia.

correlacion_return_games_won = Best_players_return_games_won['Rank_x'].corr(Best_players_return_games_won['Rank_y'])
print('La correlación entre el ranking de los jugadores y el %Return Games Won es del: ', correlacion_return_games_won)

La correlación entre el ranking de los jugadores y el %Return Games Won es del:  -0.24019223070763066


In [1258]:
## 1.3 Vamos a ver la correlación de BestTieBreaks con el el ranking de los jugadores de df_ranking.

# Lo primero que hacemos es añadir la columna ranking a BestTieBreaks para poder compararla con el ranking de los jugadores.

BestTieBreaks = BestTieBreaks.reset_index(drop=True)
BestTieBreaks['Rank'] = BestTieBreaks.index + 1
BestTieBreaks

,Player,%TieBreaks Won,Rank
0,John Isner,71.099998,1
1,Novak Djokovic,80.000000,2
2,Taylor Fritz,58.500000,3
3,Casper Ruud,61.000000,4
4,Borna Coric,66.699997,5
5,Holger Rune,63.599998,6
6,Alexander Zverev,78.599998,7
7,Andrey Rublev,61.500000,8
8,Frances Tiafoe,66.699997,9
9,Jannik Sinner,68.199997,10


In [1259]:
### Vamos a ver la correlación de BestTieBreaks con el el ranking de los jugadores de df_ranking viendo si coincide en el ranking de cada jugador en ambos dataframes.

df_ranking = df_ranking.reset_index(drop=True)
df_ranking['Rank'] = df_ranking.index + 1
df_ranking

,Player,Age,Rank
0,Carlos Alcaraz,19.0,1
1,Novak Djokovic,35.0,2
2,Stefanos Tsitsipas,24.0,3
3,Casper Ruud,24.0,4
4,Daniil Medvedev,27.0,5
...,...,...,...
1993,Stefan Vujic,24.0,1994
1994,Raffael Schaer,20.0,1995
1995,Biagio Gramaticopolo,20.0,1996
1996,Akram El Sallaly,25.0,1997


In [1260]:
### Para ver cuanto % de correlación hay entre las dos columnas vamos a hacer un merge de los dos dataframes.

df_merge = pd.merge(BestTieBreaks, df_ranking, on='Player')
df_merge

,Player,%TieBreaks Won,Rank_x,Age,Rank_y
0,John Isner,71.099998,1,37.0,46
1,Novak Djokovic,80.000000,2,35.0,2
2,Taylor Fritz,58.500000,3,25.0,10
3,Casper Ruud,61.000000,4,24.0,4
4,Borna Coric,66.699997,5,26.0,20
5,Holger Rune,63.599998,6,19.0,8
6,Alexander Zverev,78.599998,7,25.0,15
7,Andrey Rublev,61.500000,8,25.0,7
8,Frances Tiafoe,66.699997,9,25.0,14
9,Jannik Sinner,68.199997,10,21.0,11


In [1261]:
### Nos quedamos solo con aquellos que estan entre los 10 primeros en el df_merge.

Best_players_tie_breaks = df_merge[df_merge['Rank_y'] <= 10]
Best_players_tie_breaks

,Player,%TieBreaks Won,Rank_x,Age,Rank_y
1,Novak Djokovic,80.000000,2,35.0,2
2,Taylor Fritz,58.500000,3,25.0,10
3,Casper Ruud,61.000000,4,24.0,4
5,Holger Rune,63.599998,6,19.0,8
7,Andrey Rublev,61.500000,8,25.0,7


In [1262]:
### Ya sabemos cuantos jugadores hay en el top 10 del dataframe BestTieBreaks que estan en el top 10 del ranking de los jugadores. Vamos a ver el % de coincidencia.

correlacion_tie_breaks = Best_players_tie_breaks['Rank_x'].corr(Best_players_tie_breaks['Rank_y'])
print('La correlación entre el ranking de los jugadores y el %Tie Breaks Won es del: ', correlacion_tie_breaks)

La correlación entre el ranking de los jugadores y el %Tie Breaks Won es del:  0.3705363143382948


In [1263]:
## 1.4 Vamos a ver la correlación de BestDecidingSetWon con el el ranking de los jugadores de df_ranking.

# Lo primero que hacemos es añadir la columna ranking a BestDecidingSetWon para poder compararla con el ranking de los jugadores.

BestDecidingSetWon = BestDecidingSetWon.reset_index(drop=True)
BestDecidingSetWon['Rank'] = BestDecidingSetWon.index + 1
BestDecidingSetWon

,Player,%Deciding Sets Won,Rank
0,Hubert Hurkacz,69.000000,1
1,Novak Djokovic,63.599998,2
2,Stefanos Tsitsipas,73.099998,3
3,Felix Auger-Aliassime,66.699997,4
4,Carlos Alcaraz,73.900002,5
5,Andrey Rublev,65.199997,6
6,Karen Khachanov,64.699997,7
7,Arthur Rinderknech,75.000000,8
8,Jannik Sinner,63.599998,9
9,Cameron Norrie,71.000000,10


In [1264]:
### Vamos a ver la correlación de BestDecidingSetWon con el el ranking de los jugadores de df_ranking viendo si coincide en el ranking de cada jugador en ambos dataframes.

df_ranking = df_ranking.reset_index(drop=True)
df_ranking['Rank'] = df_ranking.index + 1
df_ranking

,Player,Age,Rank
0,Carlos Alcaraz,19.0,1
1,Novak Djokovic,35.0,2
2,Stefanos Tsitsipas,24.0,3
3,Casper Ruud,24.0,4
4,Daniil Medvedev,27.0,5
...,...,...,...
1993,Stefan Vujic,24.0,1994
1994,Raffael Schaer,20.0,1995
1995,Biagio Gramaticopolo,20.0,1996
1996,Akram El Sallaly,25.0,1997


In [1265]:
### Para ver cuanto % de correlación hay entre las dos columnas vamos a hacer un merge de los dos dataframes.

df_merge = pd.merge(BestDecidingSetWon, df_ranking, on='Player')
df_merge

,Player,%Deciding Sets Won,Rank_x,Age,Rank_y
0,Hubert Hurkacz,69.000000,1,26.0,9
1,Novak Djokovic,63.599998,2,35.0,2
2,Stefanos Tsitsipas,73.099998,3,24.0,3
3,Felix Auger-Aliassime,66.699997,4,22.0,6
4,Carlos Alcaraz,73.900002,5,19.0,1
5,Andrey Rublev,65.199997,6,25.0,7
6,Karen Khachanov,64.699997,7,26.0,16
7,Arthur Rinderknech,75.000000,8,27.0,72
8,Jannik Sinner,63.599998,9,21.0,11
9,Cameron Norrie,71.000000,10,27.0,12


In [1266]:
### Nos quedamos solo con aquellos que estan entre los 10 primeros en el df_merge.

Best_players_deciding_set_won = df_merge[df_merge['Rank_y'] <= 10]
Best_players_deciding_set_won

,Player,%Deciding Sets Won,Rank_x,Age,Rank_y
0,Hubert Hurkacz,69.000000,1,26.0,9
1,Novak Djokovic,63.599998,2,35.0,2
2,Stefanos Tsitsipas,73.099998,3,24.0,3
3,Felix Auger-Aliassime,66.699997,4,22.0,6
4,Carlos Alcaraz,73.900002,5,19.0,1
5,Andrey Rublev,65.199997,6,25.0,7


In [1267]:
### Ya sabemos cuantos jugadores hay en el top 10 del dataframe BestDecidingSetWon que estan en el top 10 del ranking de los jugadores. Vamos a ver el % de coincidencia.

correlacion_deciding_set_won = Best_players_deciding_set_won['Rank_x'].corr(Best_players_deciding_set_won['Rank_y'])
print('La correlación entre el ranking de los jugadores y el %Deciding Set Won es del: ', correlacion_deciding_set_won)

La correlación entre el ranking de los jugadores y el %Deciding Set Won es del:  -0.17016911869528195


In [1268]:
### Vamos a visualizar todas las correlaciones en un mismo gráfico plotly express.

# Todas las correlaciones

correlaciones = [correlacion_1st_serve_won, correlacion_return_games_won, correlacion_tie_breaks, correlacion_deciding_set_won]

# Nombres de las correlaciones

nombres = ['1st Serve Points Won', 'Return Games Won', 'Tie Breaks Won', 'Deciding Set Won']

# Creamos un dataframe con los datos

df_correlaciones = pd.DataFrame({'Correlaciones': correlaciones, 'Nombres': nombres})

# Creamos el gráfico

fig = px.bar(df_correlaciones, x='Nombres', y='Correlaciones', color='Correlaciones', color_continuous_scale='Bluered_r')
fig.show()

In [1269]:
## 2. Vamos a ver que jugadores coincide que estan en todas las listas de los mejores jugadores de cada categoría.

# Vamos a ver que jugadores estan en todas las listas de los mejores jugadores de cada categoría.

# Creamos una lista con los nombres de los jugadores de cada dataframe

lista_1st_serve_won = Best1stServePointsWon['Player'].tolist()
lista_return_games_won = BestReturnGamesWon['Player'].tolist()
lista_tie_breaks = BestTieBreaks['Player'].tolist()
lista_deciding_set_won = BestDecidingSetWon['Player'].tolist()

# Creamos una lista con los nombres de los jugadores que estan en todas las listas

lista_jugadores = []

for jugador in lista_1st_serve_won:
    if jugador in lista_return_games_won and jugador in lista_tie_breaks and jugador in lista_deciding_set_won:
        lista_jugadores.append(jugador)

print('Los jugadores que estan en todas las listas son: ', lista_jugadores)

Los jugadores que estan en todas las listas son:  ['Novak Djokovic']


In [1270]:
## 3. Vamos a ver cuanto influye la edad en el ranking de los jugadores que correlacion tiene con el ranking de los jugadores.

# Vamos a ver cuanto influye la edad en el ranking de los jugadores que correlacion tiene con el ranking de los jugadores.

edad_ranking = df_ranking[['Player', 'Age', 'Rank']]
edad_ranking

,Player,Age,Rank
0,Carlos Alcaraz,19.0,1
1,Novak Djokovic,35.0,2
2,Stefanos Tsitsipas,24.0,3
3,Casper Ruud,24.0,4
4,Daniil Medvedev,27.0,5
...,...,...,...
1993,Stefan Vujic,24.0,1994
1994,Raffael Schaer,20.0,1995
1995,Biagio Gramaticopolo,20.0,1996
1996,Akram El Sallaly,25.0,1997


In [1271]:
### Vamos a ver la correlación de la edad con el ranking de los jugadores.

correlacion_edad_ranking = edad_ranking['Age'].corr(edad_ranking['Rank'])
print('La correlación entre la edad y el ranking de los jugadores es del: ', correlacion_edad_ranking)

La correlación entre la edad y el ranking de los jugadores es del:  -0.3470325378463214


In [1272]:
### Vamos a ver la correlación de la edad con el ranking de los jugadores en un gráfico plotly.

fig = px.scatter(edad_ranking, x='Age', y='Rank', color='Rank', color_continuous_scale='Bluered_r')
fig.show()

# 3. Conclusiones

- El feature que mas correlación tiene con el ranking de los jugadores es el % Tie Breaks Won con un 38% de correlación.
- El jugador mas completo y que esta en todas las listas es Novak Djokovic.
- La edad influye en el ranking de los jugadores ya que la correlación es del 34% a favor de los mas jovenes.

# 4. Mapa Circuito ATP.

In [1273]:
### Vamos a crear un mapa del circuito ATP a lo largo del año.

calendar = pd.read_csv('Calendar_ATP.csv')
calendar

,Title,Title_URL,Label,Image,Location,tourneydates,tourneydetails,tourneydetails_URL,tourneydetails_URL1,tourneydetails2,...,tourneyresult46,tourneyresult_URL47,tourneyresult_URL48,tourneyresult49,tourneyresult50,tourneyresult_URL51,tourneyresult_URL52,tourneyresult53,tourneyresult54,tourneyresult55
0,\n United Cup\n ...,https://www.atptour.com/en/tournaments/brisban...,\nDecember 2022,https://www.atptour.com/assets/atpwt/images/to...,"\n Brisbane-Perth-Sydney, A...",\n\n\n 2022.12.29 - 2023.01...,\n SGL\...,https://www.atptour.com/en/scores/archive/bris...,https://www.atptour.com/en/scores/archive/bris...,\n Outdoor\n ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,\n Adelaide Internation...,https://www.atptour.com/en/tournaments/adelaid...,\nJanuary 2023,https://www.atptour.com/assets/atpwt/images/to...,"\n Adelaide, Australia\n ...",\n\n\n 2023.01.01 - 2023.01...,\n SGL\...,https://www.atptour.com/en/scores/archive/adel...,https://www.atptour.com/en/scores/archive/adel...,\n Outdoor\n ...,...,\n SGL\...,https://www.atptour.com/en/scores/archive/aust...,https://www.atptour.com/en/scores/archive/aust...,\n\nTotal Financial Commitment ...,\n A$34...,NaN,NaN,NaN,NaN,NaN
2,\n Davis Cup Qualifiers...,https://www.atptour.com/en/tournaments/davis-c...,\nFebruary 2023,https://www.atptour.com/assets/atpwt/images/to...,"\n Multiple Locations, Mult...",\n\n\n 2023.02.03 - 2023.02...,\n SGL\...,https://www.atptour.com/en/scores/archive/davi...,https://www.atptour.com/en/scores/archive/davi...,\n Indoor\n ...,...,\n SGL\...,https://www.atptour.com/en/scores/archive/rott...,https://www.atptour.com/en/scores/archive/rott...,\n\nTotal Financial Commitment ...,"\n €2,2...",NaN,https://www.atptour.com/en/tournaments/delray-...,\n Delray Beach Open\n ...,"\n Delray Beach, FL, U.S.A....",\n\n\n 2023.02.13 - 2023.02...
3,\n BNP Paribas Open\n ...,https://www.atptour.com/en/tournaments/indian-...,\nMarch 2023,https://www.atptour.com/assets/atpwt/images/to...,"\n Indian Wells, CA, U.S.A....",\n\n\n 2023.03.08 - 2023.03...,\n SGL\...,https://www.atptour.com/en/scores/archive/indi...,https://www.atptour.com/en/scores/archive/indi...,\n Outdoor\n ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,\n Fayez Sarofim & Co. ...,https://www.atptour.com/en/tournaments/houston...,\nApril 2023,https://www.atptour.com/assets/atpwt/images/to...,"\n Houston, TX, U.S.A.\n ...",\n\n\n 2023.04.03 - 2023.04...,\n SGL\...,NaN,NaN,\n Outdoor\n ...,...,\n SGL\...,https://www.atptour.com/en/scores/archive/barc...,https://www.atptour.com/en/scores/archive/barc...,\n\nTotal Financial Commitment ...,"\n €2,8...",http://barcelonaopenbancsabadell.com/,https://www.atptour.com/en/tournaments/munich/...,\n BMW Open by American...,"\n Munich, Germany\n ...",\n\n\n 2023.04.17 - 2023.04...
5,\n Internazionali BNL d...,https://www.atptour.com/en/tournaments/rome/41...,\nMay 2023,https://www.atptour.com/assets/atpwt/images/to...,"\n Rome, Italy\n ...",\n\n\n 2023.05.10 - 2023.05...,\n SGL\...,https://www.atptour.com/en/scores/archive/rome...,https://www.atptour.com/en/scores/archive/rome...,\n Outdoor\n ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,\n BOSS OPEN\n ...,https://www.atptour.com/en/tournaments/stuttga...,\nJune 2023,https://www.atptour.com/assets/atpwt/images/to...,"\n Stuttgart, Germany\n ...",\n\n\n 2023.06.12 - 2023.06...,\n SGL\...,https://www.atptour.com/en/scores/archive/stut...,https://www.atptour.com/en/scores/archive/stut...,\n Outdoor\n ...,...,\n SGL\...,https://www.atptour.com/en/scores/archive/mall...,https://www.atptour.com/en/scores/archive/mall...,\n\nTotal Financial Commitment ...,\n €984...,http://www.mallorca-championships.com/es/torne...,https://www.atptour.com/en/tournaments/eastbou...,\n Rothesay Internation...,"\n Eastbourne, Great Britai...",\n\n\n 2023.06.26 - 2023.07...
7,\n Wimbledon\n ...,https://www.atptour.com/en/tournaments/wimbled...,\nJuly 2023,https://www.atptour.com/assets/atpwt/images/to...,"\n London, Great Britain\n ...",\n\n\n 

In [1274]:
### Hagamos el pre-processing de los datos.

# Nos quedamos solo con los datos que nos interesan.

calendar = calendar[['Title', 'Location', 'tourneydates']]

# Cambiamos el nombre de las columnas.

calendar = calendar.rename(columns={'Title': 'Torneo', 'Label': 'Fecha', 'Location': 'Location', 'tourneydates': 'Fechas'})

In [1275]:
### Pre - processing de las columnas.

# Columnas Torneo:
# Eliminamos las \n de la columna Torneo.
# Eliminamos los espacios delante y detras

calendar['Torneo'] = calendar['Torneo'].str.replace('\n', '')
calendar['Torneo'] = calendar['Torneo'].str.strip()
calendar['Torneo'] = calendar['Torneo'].str.replace('  ', '')

# Columna Location:
# Eliminamos los espacios delante y detras

calendar['Location'] = calendar['Location'].str.strip()
calendar['Location'] = calendar['Location'].str.replace('  ', '')

# Columna Fechas:
# Eliminamos los espacios delante y detras

calendar['Fechas'] = calendar['Fechas'].str.strip()
calendar['Fechas'] = calendar['Fechas'].str.replace('  ', '')

# Columna Fechas:

# Separamos las fechas en dos columnas

calendar['Fecha Inicio'] = calendar['Fechas'].str.split('-').str[0]
calendar['Fecha Fin'] = calendar['Fechas'].str.split('-').str[1]

# Eliminamos la columna Fechas

calendar = calendar.drop(['Fechas'], axis=1)

# Columna Location: Creamos una columna con el pais que es la ultima parte de la cadena de texto.

calendar['Pais'] = calendar['Location'].str.split(',').str[-1]
calendar['Pais'] = calendar['Pais'].str.strip()
calendar['Pais'] = calendar['Pais'].str.replace('  ', '')

# Columna Location: Creamos una columna con la ciudad que es la primera parte de la cadena de texto.

calendar['Ciudad'] = calendar['Location'].str.split(',').str[0]
calendar['Ciudad'] = calendar['Ciudad'].str.strip()
calendar['Ciudad'] = calendar['Ciudad'].str.replace('  ', '')


In [1276]:
calendar

,Torneo,Location,Fecha Inicio,Fecha Fin,Pais,Ciudad
0,United Cup,"Brisbane-Perth-Sydney, Australia",2022.12.29,2023.01.08,Australia,Brisbane-Perth-Sydney
1,Adelaide International 1,"Adelaide, Australia",2023.01.01,2023.01.08,Australia,Adelaide
2,Davis Cup Qualifiers,"Multiple Locations, Multiple Locations",2023.02.03,2023.02.05,Multiple Locations,Multiple Locations
3,BNP Paribas Open,"Indian Wells, CA, U.S.A.",2023.03.08,2023.03.19,U.S.A.,Indian Wells
4,Fayez Sarofim & Co. U.S. Men's Clay Court Cham...,"Houston, TX, U.S.A.",2023.04.03,2023.04.09,U.S.A.,Houston
5,Internazionali BNL d'Italia,"Rome, Italy",2023.05.10,2023.05.21,Italy,Rome
6,BOSS OPEN,"Stuttgart, Germany",2023.06.12,2023.06.18,Germany,Stuttgart
7,Wimbledon,"London, Great Britain",2023.07.03,2023.07.16,Great Britain,London
8,National Bank Open Presented by Rogers,"Toronto, Canada",2023.08.07,2023.08.13,Canada,Toronto
9,Davis Cup Finals Group Stage,"Multiple Locations, Multiple Locations",2023.09.12,2023.09.17,Multiple Locations,Multiple Locations


In [1277]:
### Las columnas Fecha Inicio y Fecha Fin son de tipo object. Vamos a cambiarlas a tipo datetime.

calendar['Fecha Inicio'] = pd.to_datetime(calendar['Fecha Inicio'])
calendar['Fecha Fin'] = pd.to_datetime(calendar['Fecha Fin'])

In [1278]:
calendar

,Torneo,Location,Fecha Inicio,Fecha Fin,Pais,Ciudad
0,United Cup,"Brisbane-Perth-Sydney, Australia",2022-12-29,2023-01-08,Australia,Brisbane-Perth-Sydney
1,Adelaide International 1,"Adelaide, Australia",2023-01-01,2023-01-08,Australia,Adelaide
2,Davis Cup Qualifiers,"Multiple Locations, Multiple Locations",2023-02-03,2023-02-05,Multiple Locations,Multiple Locations
3,BNP Paribas Open,"Indian Wells, CA, U.S.A.",2023-03-08,2023-03-19,U.S.A.,Indian Wells
4,Fayez Sarofim & Co. U.S. Men's Clay Court Cham...,"Houston, TX, U.S.A.",2023-04-03,2023-04-09,U.S.A.,Houston
5,Internazionali BNL d'Italia,"Rome, Italy",2023-05-10,2023-05-21,Italy,Rome
6,BOSS OPEN,"Stuttgart, Germany",2023-06-12,2023-06-18,Germany,Stuttgart
7,Wimbledon,"London, Great Britain",2023-07-03,2023-07-16,Great Britain,London
8,National Bank Open Presented by Rogers,"Toronto, Canada",2023-08-07,2023-08-13,Canada,Toronto
9,Davis Cup Finals Group Stage,"Multiple Locations, Multiple Locations",2023-09-12,2023-09-17,Multiple Locations,Multiple Locations


In [1287]:
### Tenemos las latitudes y longitudes de las ciudades que vamos a utilizar en el mapa.

# Creamos un diccionario con las ciudades y sus coordenadas.

diccionario_ciudades = {'Brisbane': [-27.46794, 153.02809],
                        'Adelaide': [-34.92866, 138.60008],
                        'Miami': [25.76168, -80.19179],
                        'Houston': [29.7604, -95.3698],
                        'Rome': [41.90278, 12.49636],
                        'Stuttgart': [48.77585, 9.18293],
                        'London': [51.50735, -0.12776],
                        'Toronto': [43.6532, -79.38318],
                        'Shanghai': [31.23039, 121.4737],
                        'Metz': [49.11914, 6.17539],
                        }

In [1288]:
# Mapa del circuito ATP.

import folium as folium

mapa = folium.Map(location=[0, 0], zoom_start=2)

for ciudad, coord in diccionario_ciudades.items():
    folium.Marker(location=coord, popup=ciudad).add_to(mapa)
    folium.CircleMarker(location=coord, radius=10, color='red', fill=True).add_to(mapa)

mapa